# Experiment 1: Datacenter Sustainability

> Trade-off energy consumption vs. risk of SLA non-fulfillment: "more resources guarantee better that there will be no performance or availability hiccup"
Argument: if we were to try different extremes, we would get … trade-offs - evaluate optimal Pareto frontier. Comment on the shape of the Pareto frontier, what happens with the scenarios far off the Pareto optimal frontier

### Replication
The experimental results are obtained by running the following command:
```bash
bin/radice run -r 4096 -p 80 portfolios/baseline.yml -P portfolio=baseline
```

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.optimize
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import radice

FIGURE_PATH = 'figures/01-sustainability'

os.makedirs(FIGURE_PATH, exist_ok=True)
radice.set()

In [ ]:
electricity_price = pd.read_csv('electricity-price.csv')
electricity_price['timestamp'] = pd.to_datetime(electricity_price['timestamp'], unit='ms')
electricity_price

In [ ]:
electricity_price.groupby(electricity_price['timestamp'].dt.year.rename("year")).describe()

In [ ]:
fig, ax = plt.subplots(1, figsize=radice.figsize(1.5))

sns.lineplot(data=electricity_price, x='timestamp', y='price_per_mwh', ax=ax, linewidth=1)

ax.set_ylabel("Electricity Price\n[€ per MWh]")
ax.set_xlabel("")

xmin=pd.to_datetime('2020-01-01', format='%Y-%m-%d')
xmax=pd.to_datetime('2021-12-20', format='%Y-%m-%d')
ax.set_xlim(left=xmin, right=xmax)
ax.set_ylim(bottom=0, top=400)
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b '%y"))

# ax.tick_params(axis='y', pad=0)

fig.savefig(os.path.join(FIGURE_PATH, 'electricity-price.pdf'))

In [ ]:
co2_price = pd.read_csv('co2-price.csv')
co2_price['timestamp'] = pd.to_datetime(co2_price['timestamp'])
co2_price

In [ ]:
co2_price.groupby(co2_price['timestamp'].dt.year.rename("year")).describe()

In [ ]:
fig, ax = plt.subplots(1, figsize=radice.figsize(1.5))

sns.lineplot(data=co2_price, x='timestamp', y='price_per_t', linewidth=1)

ax.set_ylabel("CO2 Price\n[€ per tonne]")
ax.set_xlabel("")

xmin=pd.to_datetime('2020-01-01', format='%Y-%m-%d')
xmax=pd.to_datetime('2021-12-01', format='%Y-%m-%d')
ax.set_xlim(left=xmin, right=xmax)

ax.xaxis.set_major_formatter(mdates.DateFormatter("%b '%y"))

fig.savefig(os.path.join(FIGURE_PATH, 'co2-price.pdf'))

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=radice.figsize(4), sharex=True)

sns.lineplot(data=electricity_price, x='timestamp', y='price_per_mwh', ax=ax[0], linewidth=1)

ax[0].set_ylabel("Electricity Price\n[€ per MWh]")
ax[0].set_xlabel("")
ax[0].set_ylim(bottom=0, top=440)
ax[0].set_yticks([0, 100, 250, 400])

sns.lineplot(data=co2_price, x='timestamp', y='price_per_t', linewidth=1, ax=ax[1])

ax[1].set_ylabel("CO2 Price\n[€ per tonne]")
ax[1].set_xlabel("")
ax[1].set_ylim(bottom=0)

xmin=pd.to_datetime('2020-01-01', format='%Y-%m-%d')
xmax=pd.to_datetime('2021-12-31', format='%Y-%m-%d')
ax[1].set_xlim(left=xmin, right=xmax)
ax[1].xaxis.set_major_formatter(mdates.DateFormatter("%b '%y"))

fig.align_ylabels(ax)
fig.savefig(os.path.join(FIGURE_PATH, 'resource-price.pdf'))

## Impact of Electricity/CO2 Prices

In [ ]:
risk = pd.read_parquet('data/risk/portfolio=baseline/topology=base', columns=['seed', 'timestamp', 'id', 'cost', 'value'], read_dictionary=['id'], buffer_size=4096)
risk

In [ ]:
risk_pm_2021_oct = radice.compute_monthly_risk(risk)
risk_pm_2021_oct.groupby(["id"])['cost'].describe()

In [ ]:
# Risk profile on October 7, 2021 where the electricity price is over 300 euros
fig, ax = radice.plot_risk_factors_horizontal(risk_pm_2021_oct, height=2.1)

fig.savefig(os.path.join(FIGURE_PATH, 'risk-profile-2021-oct.pdf'))

In [ ]:
fig, ax = plt.subplots(1, figsize=(6.2, 1.3))

data = radice.compute_monthly_risk(risk_pm_2021_oct.assign(group=risk_pm_2021_oct['id'].str.split(':').str[0]), keys=['group'])

sns.boxplot(x="cost", y="group", data=data, ax=ax, showfliers=False, showmeans=True, order=['customer', 'company', 'society'], color='C0')

ax.set_xlabel("Incurred Monthly Cost (€)", usetex=False)
ax.set_yticklabels(["Customer", "Company", "Society"])
ax.set_ylabel("Risk Class")

fig.savefig(os.path.join(FIGURE_PATH, 'risk-classes-2021-oct.pdf'))

In [ ]:
# Risk profile of last year
risk_pm_2020 = radice.compute_monthly_risk(radice.adjust_env(risk, 30, 33))
risk_pm_2020.groupby(["id"])['cost'].describe()

In [ ]:
fig, ax = radice.plot_risk_factors_horizontal(risk_pm_2020)
fig.savefig(os.path.join(FIGURE_PATH, 'risk-profile-2020.pdf'))

In [ ]:
data = pd.concat([
    risk_pm_2020.assign(year=2020),
    risk_pm_2021_oct.assign(year=2021)
])

fig, ax = radice.plot_risk_factors_horizontal(data, hue='year')

legend = ax[0].get_legend()
legend.set_title("Scenario")
legend.get_texts()[0].set_text('October 2020')
legend.get_texts()[1].set_text('October 2021')

fig.savefig(os.path.join(FIGURE_PATH, 'risk-comparison.pdf'))

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=radice.figsize(2.1), sharey=True, gridspec_kw={'width_ratios': [3, 4, 1]})

cx_order = ['customer:availability', 'customer:latency', 'customer:cpu_interference']
cx_labels = ['Availability', 'Scalability', 'QoS']

co_order = ['company:power', 'company:co2', 'company:host_saturation', 'company:host_imbalance']
co_labels = ['Electricity\nDemand', 'CO2\nEmissions', 'Resource\nSaturation', 'Resource\nImbalance']

soc_order = ['society:co2']
soc_labels = ['CO2\nEmissions']

sns.barplot(x="id", y="cost", data=risk_pm_2020[risk_pm_2020['id'].str.startswith('customer')], ax=ax[0], order=cx_order, edgecolor='black', color='C0', zorder=2, ci=None)
sns.barplot(x="id", y="cost", data=risk_pm_2021_oct[risk_pm_2021_oct['id'].str.startswith('customer')], ax=ax[0], order=cx_order, edgecolor='black', color='C1')

ax[0].set_ylabel("Risk per Month (€)")
ax[0].set_xticklabels(cx_labels)
ax[0].set_xlabel("")
ax[0].set_title("Customer")
ax[0].yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))

patch_2020 = mpatches.Patch(facecolor='C0', edgecolor='black', label='October 2020')
patch_2021 = mpatches.Patch(facecolor='C1', edgecolor='black', label='October 2021')

legend = ax[1].legend(title='Scenario', handles=[patch_2020, patch_2021])

sns.barplot(x="id", y="cost", data=risk_pm_2020[risk_pm_2020['id'].str.startswith('company')], ax=ax[1], order=co_order, edgecolor='black', color='C0', zorder=2, ci=None)
sns.barplot(x="id", y="cost", data=risk_pm_2021_oct[risk_pm_2021_oct['id'].str.startswith('company')], ax=ax[1], order=co_order, edgecolor='black', color='C1')

ax[1].set_ylabel("")
ax[1].set_xticklabels(co_labels)
ax[1].set_xlabel("")
ax[1].set_title("Company")
ax[0].legend([],[], frameon=False)

sns.barplot(x="id", y="cost", data=risk_pm_2020[risk_pm_2020['id'].str.startswith('society')], ax=ax[2], order=soc_order, edgecolor='black', color='C0', zorder=2, ci=None)
sns.barplot(x="id", y="cost", data=risk_pm_2021_oct[risk_pm_2021_oct['id'].str.startswith('society')], ax=ax[2], order=soc_order, edgecolor='black', color='C1')

ax[2].set_ylabel("")
ax[2].set_xticklabels(soc_labels)
ax[2].set_xlabel("")
ax[2].set_title("Society")
ax[2].legend([],[], frameon=False)

fig.align_xlabels(ax)

for x in ax:
    radice.align_labels(x, va='center')

fig.savefig(os.path.join(FIGURE_PATH, 'risk-comparison.pdf'))

In [ ]:
# Risk profile of this month
risk_pm_2021_nov = radice.compute_monthly_risk(radice.adjust_env(risk, 170, 60))
risk_pm_2021_nov.groupby(["id"])['cost'].describe()

In [ ]:
fig, ax = radice.plot_risk_factors_horizontal(risk_pm_2021_nov)
fig.savefig(os.path.join(FIGURE_PATH, 'risk-profile-2021-nov.pdf'))

In [ ]:
risk_pm_600 = radice.compute_monthly_risk(radice.adjust_env(risk, 600, 128))
risk_pm_600.groupby(["id"])['cost'].describe()

In [ ]:
fig, _ = radice.plot_risk_factors_horizontal(risk_pm_600)

In [ ]:
risk_3000 = radice.adjust_env(risk, 3000, 640)
risk_pm_3000 = radice.compute_monthly_risk(risk_3000)
risk_pm_3000.groupby(["id"])['cost'].describe()

In [ ]:
fig, _ = radice.plot_risk_factors_horizontal(risk_pm_3000)

In [ ]:
risk_google = radice.adjust_env(risk, pue = 1.10)
risk_pm_google = radice.compute_monthly_risk(risk_google)
risk_pm_google.groupby(["id"])['cost'].describe()

In [ ]:
fig, _ = radice.plot_risk_factors_horizontal(risk_pm_google)

In [ ]:
data = pd.concat([
    risk_pm_2020.assign(scenario='2020'),
    risk_pm_2021_oct.assign(scenario='2021-oct'),
    risk_pm_2021_nov.assign(scenario='2021-nov'),
    risk_pm_600.assign(scenario='prices-2x'),
    risk_pm_3000.assign(scenario='prices-10x'),
])

data = pd.concat([
    radice.adjust_pue(data, pue=1.1).assign(pue='1.1'),
    data.assign(pue='1.57'),
    radice.adjust_pue(data, pue=2.5).assign(pue='2.5'),
])

baseline = radice.compute_monthly_risk(risk_pm_2020, keys=[])['cost'].mean()
data = data.groupby(['seed', 'scenario', 'pue', 'month'])['cost'].sum()
data = data.to_frame().reset_index()
data['pct'] = data['cost'] / baseline
data

In [ ]:
data.groupby(['scenario', 'pue'])['cost'].mean()

In [ ]:
fig, ax = plt.subplots(1, figsize=radice.figsize(2.3))

order=['2020', '2021-nov', '2021-oct', 'prices-2x', 'prices-10x']

sns.barplot(data=data, x='pct', y='scenario', hue='pue', ax=ax, edgecolor='black', order=order)

ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.set_xlabel("Relative Risk")
ax.set_ylabel("Scenario")
ax.set_yticklabels([
  'Last year',
  'This year',
  'This year\n(Peak)',
  'Prices 2x',
  'Prices 10x'
])

legend = ax.get_legend()
legend.set_title("PUE")
labels = ['Google (1.1)', 'Global Average (1.57)', 'High (2.5)']

for i, txt in enumerate(legend.get_texts()):
    txt.set_text(labels[i])

ax.axvline(1, color='w', alpha=0.9, linestyle='dashed', lw=1.2)

radice.align_labels(ax, axis='y', ma='center', offset=-8)

fig.savefig(os.path.join(FIGURE_PATH, 'electricity-price-impact.pdf'))

## Impact of Topology Size

In [ ]:
risk_12_pm = pd.read_parquet('data/risk/portfolio=baseline/topology=base-12.5%', columns=['seed', 'timestamp', 'id', 'cost'])
risk_12_pm = radice.compute_monthly_risk(risk_12_pm)
risk_12_pm.groupby(["id"])['cost'].describe()

In [ ]:
risk_25_pm = pd.read_parquet('data/risk/portfolio=baseline/topology=base-25%', columns=['seed', 'timestamp', 'id', 'cost'])
risk_25_pm = radice.compute_monthly_risk(risk_25_pm)
risk_25_pm.groupby(["id"])['cost'].describe()

In [ ]:
fig, _ = radice.plot_risk_factors_horizontal(risk_25_pm)
fig.savefig(os.path.join(FIGURE_PATH, 'risk-profile-topology-25.pdf'))

In [ ]:
risk_37_pm = pd.read_parquet('data/risk/portfolio=baseline/topology=base-37.5%', columns=['seed', 'timestamp', 'id', 'cost'])
risk_37_pm = radice.compute_monthly_risk(risk_37_pm)
risk_37_pm.groupby(["id"])['cost'].describe()

In [ ]:
risk_50_pm = pd.read_parquet('data/risk/portfolio=baseline/topology=base-50%', columns=['seed', 'timestamp', 'id', 'cost'])
risk_50_pm = radice.compute_monthly_risk(risk_50_pm)
risk_50_pm.groupby(["id"])['cost'].describe()

In [ ]:
fig, ax = radice.plot_risk_factors_horizontal(risk_50_pm)
fig.savefig(os.path.join(FIGURE_PATH, 'risk-profile-topology-50.pdf'))

In [ ]:
risk_75_pm = pd.read_parquet('data/risk/portfolio=baseline/topology=base-75%', columns=['seed', 'timestamp', 'id', 'cost'])
risk_75_pm = radice.compute_monthly_risk(risk_75_pm)
risk_75_pm.groupby(["id"])['cost'].describe()

In [ ]:
risk_90_pm = pd.read_parquet('data/risk/portfolio=baseline/topology=base-90%', columns=['seed', 'timestamp', 'id', 'cost'])
risk_90_pm = radice.compute_monthly_risk(risk_90_pm)
risk_90_pm.groupby(["id"])['cost'].describe()

In [ ]:
risk_150_pm = pd.read_parquet('data/risk/portfolio=baseline/topology=base-150%', columns=['seed', 'timestamp', 'id', 'cost'])
risk_150_pm = radice.compute_monthly_risk(risk_150_pm)
risk_150_pm.groupby(["id"])['cost'].describe()

In [ ]:
risk_200_pm = pd.read_parquet('data/risk/portfolio=baseline/topology=base-200%', columns=['seed', 'timestamp', 'id', 'cost'])
risk_200_pm = radice.compute_monthly_risk(risk_200_pm)
risk_200_pm.groupby(["id"])['cost'].describe()

In [ ]:
fig, _ = radice.plot_risk_factors_horizontal(risk_200_pm)
fig.savefig(os.path.join(FIGURE_PATH, 'risk-profile-topology-200.pdf'))

In [ ]:
risk_300_pm = pd.read_parquet('data/risk/portfolio=baseline/topology=base-300%', columns=['seed', 'timestamp', 'id', 'cost'])
risk_300_pm = radice.compute_monthly_risk(risk_300_pm)
risk_300_pm.groupby(["id"])['cost'].describe()

In [ ]:
risk_400_pm = pd.read_parquet('data/risk/portfolio=baseline/topology=base-400%', columns=['seed', 'timestamp', 'id', 'cost'])

risk_400_pm = radice.compute_monthly_risk(risk_400_pm)
risk_400_pm.groupby(["id"])['cost'].describe()

In [ ]:
fig, _ = radice.plot_risk_factors_horizontal(risk_400_pm)
fig.savefig(os.path.join(FIGURE_PATH, 'risk-profile-topology-400.pdf'))

In [ ]:
data = pd.concat([
    risk_12_pm.assign(scale=0.125),
    risk_25_pm.assign(scale=0.25),
    risk_37_pm.assign(scale=0.375),
    risk_50_pm.assign(scale=0.5),
    risk_75_pm.assign(scale=0.75),
    risk_90_pm.assign(scale=0.9),
    risk_pm_2021_oct.assign(scale=1),
    risk_150_pm.assign(scale=1.5),
    risk_200_pm.assign(scale=2),
    risk_300_pm.assign(scale=3),
    risk_400_pm.assign(scale=4),
])

data = pd.concat([
    radice.adjust_pue(data, pue=1.1).assign(pue='1.1'),
    data.assign(pue='1.57'),
    radice.adjust_pue(data, pue=2.5).assign(pue='2.5'),
])

baseline = radice.compute_monthly_risk(risk, keys=[])['cost'].mean()

data = data.groupby(['seed', 'scale', 'pue', 'month'])['cost'].sum().reset_index()
data['increase'] = data['cost'] / radice.compute_monthly_risk(risk, keys=[])['cost'].mean()
data

In [ ]:
fig, ax = plt.subplots(1, figsize=radice.figsize(2))

order=[0.25, 0.5, 0.75, 1.0, 2.0, 4.0]

sns.barplot(data=data, x='increase', y='scale', hue='pue', orient='h', ax=ax, edgecolor='black', order=order)

ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.set_xlabel("Relative Risk Increase")
ax.set_ylabel("Topology Scale")
ax.set_yticklabels([
    '0.25x',
    '0.5x',
    '0.75x',
    'Baseline',
    '2x',
    '4x'
])

legend = ax.get_legend()
legend.set_title("PUE")
labels = ['Google (1.1)', 'Global Average (1.57)', 'High (2.5)']

for i, txt in enumerate(legend.get_texts()):
    txt.set_text(labels[i])

ax.axvline(1, color='w', alpha=0.9, linestyle='dashed', lw=1.2)

fig.savefig(os.path.join(FIGURE_PATH, 'topology-scale-impact.pdf'))

In [ ]:
data_m = data # [data['pue'] == '1.57']
data_m = data_m.groupby(['pue', 'scale']).mean().reset_index()
data_m

In [ ]:
fig, ax = plt.subplots(1, figsize=radice.figsize(2))

def f(x, mu, sigma, a, b):
    return scipy.stats.lognorm.pdf(x, sigma, scale = np.exp(mu)) + a * x + b

for pue, group in data_m.groupby(['pue']):
    x = group['scale']
    y = group['increase']
    xp = np.linspace(x.min(), x.max(), 100)

    # Fit the curve according to our function
    params, extras = scipy.optimize.curve_fit(f, x, y)
    # Find minimum value
    xmin = scipy.optimize.fmin(lambda x: f(x, *params), 1)[0]

    sns.scatterplot(x=[xmin], y=[f(xmin, *params) * 9321.649190], ax=ax, edgecolor='black', zorder=150, marker='d')
    sns.lineplot(x=xp, y=f(xp, *params) * 9321.649190, ax=ax, linewidth=0.8)

sns.scatterplot(data=data_m, x='scale', y='cost', hue='pue', ax=ax, edgecolor='black', zorder=100)

ax.xaxis.set_major_formatter('{x}x')
ax.set_xlabel("Topology Scale")
# ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))
ax.set_ylabel("Risk per Month (€)")
ax.set_ylim(bottom=0, top=50000)
#ax.ticklabel_format(useOffset=True, axis='y', style='sci', scilimits=(0, 0), useMathText=True)

legend_elements = [
    Line2D([0], [0], marker='o', linestyle='None', markerfacecolor='C0', markeredgecolor='black', markersize=8, label='Google (1.1)'),
    Line2D([0], [0], marker='o', linestyle='None', markerfacecolor='C1', markeredgecolor='black', markersize=8, label='Global Average (1.57)'),
    Line2D([0], [0], marker='o', linestyle='None', markerfacecolor='C2', markeredgecolor='black', markersize=8, label='High (2.5)'),
]

legend = ax.legend(handles=legend_elements, title="PUE", loc='upper center', ncol=3)

fig.savefig(os.path.join(FIGURE_PATH, 'topology-scale-opt.pdf'))

In [ ]:
fig, ax = plt.subplots(1, figsize=radice.figsize(2))

scale_opt_f = radice.compute_monthly_risk(risk, keys=[])['cost'].mean()

def f(x, mu, sigma, a, b):
    return scipy.stats.lognorm.pdf(x, sigma, scale = np.exp(mu)) + a * x + b

for pue, group in data_m.groupby(['pue']):
    x = group['scale']
    y = group['increase']
    xp = np.linspace(x.min(), x.max(), 100)

    # Fit the curve according to our function
    params, extras = scipy.optimize.curve_fit(f, x, y)
    # Find minimum value
    xmin = scipy.optimize.fmin(lambda x: f(x, *params), 1)[0]

    sns.scatterplot(x=[xmin], y=[(f(xmin, *params) * 9321.649190) / scale_opt_f], ax=ax, edgecolor='black', zorder=150, marker='d')
    sns.lineplot(x=xp, y=(f(xp, *params) * 9321.649190) / scale_opt_f, ax=ax, linewidth=0.8)

sns.scatterplot(data=data_m, x='scale', y='increase', hue='pue', ax=ax, edgecolor='black', zorder=100)

ax.xaxis.set_major_formatter('{x}x')
ax.set_xlabel("Topology Scale")
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
#ax.yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))
ax.set_ylabel("Relative Risk")
ax.set_ylim(bottom=0)
#ax.ticklabel_format(useOffset=True, axis='y', style='sci', scilimits=(0, 0), useMathText=True)

legend_elements = [
    Line2D([0], [0], marker='o', linestyle='None', markerfacecolor='C0', markeredgecolor='black', markersize=8, label='Google (1.1)'),
    Line2D([0], [0], marker='o', linestyle='None', markerfacecolor='C1', markeredgecolor='black', markersize=8, label='Global Average (1.57)'),
    Line2D([0], [0], marker='o', linestyle='None', markerfacecolor='C2', markeredgecolor='black', markersize=8, label='High (2.5)'),
]
    
legend = ax.legend(handles=legend_elements, title="PUE", loc='upper center', ncol=3)

fig.savefig(os.path.join(FIGURE_PATH, 'topology-scale-opt-rel.pdf'))

## Impact of CO2 pricing

In [ ]:
def map_group(identifier):
    if identifier == 'company:co2' or identifier == 'society:co2':
        return identifier
    else:
        return 'other'

operating_margin = 1.20

group_costs = risk_pm_2021_oct.assign(group=risk_pm_2021_oct['id'].apply(map_group))
group_costs = group_costs.groupby(['group', 'seed', 'month'])['cost'].sum().reset_index()
group_costs = pd.concat([
    group_costs,
    group_costs.groupby(['seed', 'month'])['cost'].sum().reset_index().assign(group='all'),
    (group_costs.groupby(['seed', 'month'])['cost'].sum() * operating_margin).reset_index().assign(group='revenue')
])
group_costs['fraction'] = group_costs['cost'] / group_costs.loc[group_costs['group'] == 'all', 'cost'].mean()
group_costs

In [ ]:
co2_cost = risk_pm_2021_oct
co2_cost = pd.concat([
    radice.adjust_co2(co2_cost, co2_price=64 * 2).assign(co2_price=64 * 2),
    radice.adjust_co2(co2_cost, co2_price=64 * 5).assign(co2_price=64 * 5),
    radice.adjust_co2(co2_cost, co2_price=64 * 10).assign(co2_price=64 * 10),
])
co2_cost = pd.concat([
    co2_cost.groupby(['co2_price', 'seed', 'month'])['cost'].sum().reset_index().assign(group='all'),
    co2_cost[co2_cost['id'] == 'company:co2'].groupby(['co2_price', 'seed', 'month'])['cost'].sum().reset_index().assign(group='company:co2'),
])
co2_cost['fraction'] = co2_cost['cost'] / group_costs.loc[group_costs['group'] == 'all', 'cost'].mean()
co2_cost

In [ ]:
fig, ax = plt.subplots(1, figsize=radice.figsize(2))

order = ['revenue', 'all', 'other', 'society:co2', 'company:co2']

bar = sns.barplot(data=group_costs, x='fraction', y='group', ax=ax, edgecolor='black', order=order, ci=None, zorder=2, color='C0')
bar.patches[0].set_hatch('////')

sns.barplot(data=co2_cost[co2_cost['co2_price'] == 64 * 2], x='fraction', y='group', ax=ax, edgecolor='black', order=order, ci=None, zorder=1.9, color='C1')

sns.barplot(data=co2_cost[co2_cost['co2_price'] == 64 * 5], x='fraction', y='group', ax=ax, edgecolor='black', order=order, ci=None, zorder=1.8, color='C2')
sns.barplot(data=co2_cost[co2_cost['co2_price'] == 64 * 10], x='fraction', y='group', ax=ax, edgecolor='black', order=order, ci=None, color='C3')

ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.set_xlabel("Fraction of Monthly Costs")
ax.set_ylabel("Cost Factor")
ax.set_yticklabels([
    'Revenue',
    'Total Costs',
    'Other Costs',
    'CO2 (Society)',
    'CO2 (Company)',
], usetex=False)

patch_baseline = mpatches.Patch(facecolor='C0', edgecolor='black', label='Baseline\n(€ 64 per tCO2)')
patch_2x = mpatches.Patch(facecolor='C1', edgecolor='black', label='2x CO2 Price')
patch_5x = mpatches.Patch(facecolor='C2', edgecolor='black', label='5x CO2 Price')
patch_10x = mpatches.Patch(facecolor='C3', edgecolor='black', label='10x CO2 Price')

legend = ax.legend(title='CO2 Price', handles=[patch_baseline, patch_2x, patch_5x, patch_10x])
txt = legend.get_texts()
txt[0].set_ma('center')

radice.align_labels(ax, ha='right', axis='y', offset=-8)

fig.savefig(os.path.join(FIGURE_PATH, 'co2-price-impact.pdf'))

## Environmental Impact

In [ ]:
df = pd.read_parquet('data/risk', columns=['value'], buffer_size=4096, filters=[('id', '=', 'company:power')])
df['value'].sum()

In [ ]:
df = pd.read_parquet('data/risk', columns=['cost'], buffer_size=4096, filters=[('id', '=', 'company:power')])
df['cost'].sum()

In [ ]:
df = pd.read_parquet('data/risk', columns=['value'], buffer_size=4096, filters=[('id', '=', 'company:co2')])
df['value'].sum()

In [ ]:
df = pd.read_parquet('data/risk/portfolio=baseline',  columns=['timestamp', 'topology', 'seed'], buffer_size=4096, filters=[('id', '=', 'company:power')])
(df.groupby(['topology', 'seed'])['timestamp'].max() - pd.Timestamp(0, tz='UTC')).dt.total_seconds().sum()

In [ ]:
df = pd.read_parquet('data/risk/portfolio=phenomena',  columns=['timestamp', 'scenario', 'seed'], buffer_size=4096, filters=[('id', '=', 'company:power')])
(df.groupby(['scenario', 'seed'])['timestamp'].max() - pd.Timestamp(0, tz='UTC')).dt.total_seconds().sum()

In [ ]:
df = pd.read_parquet('data/risk/portfolio=topology-opt',  columns=['timestamp', 'candidate', 'seed'], buffer_size=4096, filters=[('id', '=', 'company:power')])
(df.groupby(['candidate', 'seed'])['timestamp'].max() - pd.Timestamp(0, tz='UTC')).dt.total_seconds().sum()

In [ ]:
df = pd.read_parquet('data/risk/portfolio=workload',  columns=['timestamp', 'workload', 'seed'], buffer_size=4096, filters=[('id', '=', 'company:power')])
(df.groupby(['workload', 'seed'])['timestamp'].max() - pd.Timestamp(0, tz='UTC')).dt.total_seconds().sum()

In [ ]:
df = pd.read_parquet('data/risk/portfolio=workload-opt',  columns=['timestamp', 'topology', 'seed'], buffer_size=4096, filters=[('id', '=', 'company:power')])
(df.groupby(['topology', 'seed'])['timestamp'].max() - pd.Timestamp(0, tz='UTC')).dt.total_seconds().sum()

In [ ]:
df = pd.read_parquet('data/risk/portfolio=scheduler',  columns=['timestamp', 'scheduler', 'seed'], buffer_size=4096, filters=[('id', '=', 'company:power')])
(df.groupby(['scheduler', 'seed'])['timestamp'].max() - pd.Timestamp(0, tz='UTC')).dt.total_seconds().sum()

In [ ]:
df = pd.read_parquet('data/risk/portfolio=scheduler-opt',  columns=['timestamp', 'candidate', 'seed'], buffer_size=4096, filters=[('id', '=', 'company:power')])
(df.groupby(['candidate', 'seed'])['timestamp'].max() - pd.Timestamp(0, tz='UTC')).dt.total_seconds().sum()
